In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import make_scorer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns


from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

import pylab as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore') 
plt.rcParams['figure.figsize'] = (10, 8)
# Any results you write to the current directory are saved as output.

In [2]:
df = pd.read_csv('epldata_final.csv')

In [3]:
df.head()

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.10%,264,3.0,Chile,0,4,1,1,0
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2.0,Germany,0,4,1,1,0
2,Petr Cech,Arsenal,35,GK,4,7.0,1529,5.5,5.90%,134,2.0,Czech Republic,0,6,1,1,0
3,Theo Walcott,Arsenal,28,RW,1,20.0,2393,7.5,1.50%,122,1.0,England,0,4,1,1,0
4,Laurent Koscielny,Arsenal,31,CB,3,22.0,912,6.0,0.70%,121,2.0,France,0,4,1,1,0


In [4]:
def category(x):
    if 0 <= x < 15: return 1
    elif 15 <= x < 50: return 2
    else: return 3

In [5]:
# делаем задачу классификации
df['market_value'] =  df['market_value'].apply(category)

In [6]:
df['region'].fillna((df['region'].median()), inplace=True)
#df.isnull().any()

In [7]:
def create_df(data_fram, feature_list):
    out = data_fram
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
    out.drop(feature_list, axis = 1, inplace = True)
    return out

df1 = df
features = ['club', 'position', 'nationality', 'position_cat', 'region']
for feature in features:
    df1[feature] = df1[feature].astype('object')
df1 = create_df(df1, features)
df1.head()

,name,age,market_value,page_views,fpl_value,fpl_sel,fpl_points,new_foreign,age_cat,club_id,...,nationality_Venezuela,nationality_Wales,position_cat_1,position_cat_2,position_cat_3,position_cat_4,region_1.0,region_2.0,region_3.0,region_4.0
0,Alexis Sanchez,28,3,4329,12.0,17.10%,264,0,4,1,...,0,0,1,0,0,0,0,0,1,0
1,Mesut Ozil,28,3,4395,9.5,5.60%,167,0,4,1,...,0,0,1,0,0,0,0,1,0,0
2,Petr Cech,35,1,1529,5.5,5.90%,134,0,6,1,...,0,0,0,0,0,1,0,1,0,0
3,Theo Walcott,28,2,2393,7.5,1.50%,122,0,4,1,...,0,0,1,0,0,0,1,0,0,0
4,Laurent Koscielny,31,2,912,6.0,0.70%,121,0,4,1,...,0,0,0,0,1,0,0,1,0,0


In [8]:
y = df1['market_value']
df1 = df1.drop(['name', 'market_value', 'fpl_sel'], axis=1)

In [9]:
from sklearn import preprocessing
# normalize the data attributes
normalized_X = preprocessing.normalize(df1)
# standardize the data attributes
standardized_X = preprocessing.scale(df1)

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(df1, y, test_size = 0.3, random_state=17)

In [11]:
clf_tree = DecisionTreeClassifier(max_depth=4, random_state=17)
clf_tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [12]:
predicted = clf_tree.predict(X_valid)
accuracy = accuracy_score(y_valid, predicted)
accuracy

0.8129496402877698

In [13]:
dict_model_result = {}

# bagging over trees
bc = BaggingClassifier(base_estimator=clf_tree, n_estimators=40, random_state=17)
bc.fit(X_train, y_train)
bc_preds = bc.predict(X_valid)
dict_model_result['Accuracy Bagging on valid set'] = accuracy_score(y_valid, bc_preds)

In [15]:
params = {'n_estimators': [40, 42], 'base_estimator__max_leaf_nodes':[10, 15], 'base_estimator__max_depth':[4, 5, 6]}

# Grid Search to determine best parameters
bc_grid = GridSearchCV(estimator=bc, param_grid=params, scoring='accuracy', cv=5, n_jobs=-1)
bc_grid.fit(X_train, y_train)
best_params = bc_grid.best_params_
print('лучшее значение скора на кросс-валидации -', bc_grid.best_score_)
print('лучшие характеристики -',best_params)

лучшее значение скора на кросс-валидации - 0.8633540372670807
лучшие характеристики - {'base_estimator__max_depth': 5, 'base_estimator__max_leaf_nodes': 15, 'n_estimators': 40}


In [16]:
# Random Forest
rfc = RandomForestClassifier(max_depth=4,  max_features=None, random_state=17)
rfc.fit(X_train, y_train)
rfc_preds = rfc.predict(X_valid)
dict_model_result['Accuracy Random Forest on valid set'] = accuracy_score(y_valid, rfc_preds)

In [17]:
max_depth_values = range(4, 8)
forest_params = {'max_depth': max_depth_values}

random_forest_classifier = RandomForestClassifier(class_weight='balanced', random_state = 17)
random_forest_classifier_grid = GridSearchCV(random_forest_classifier, param_grid=forest_params, scoring='accuracy', cv = 5)
random_forest_classifier_grid.fit(X_train, y_train)
print('лучшее значение скора на кросс-валидации -', random_forest_classifier_grid.best_score_)

лучшее значение скора на кросс-валидации - 0.8416149068322981


In [18]:
logit = LogisticRegression(random_state=17, multi_class='ovr', class_weight='balanced')
logit.fit(X_train, y_train)
log_predict = logit.predict(X_valid)
dict_model_result['Accuracy Logistic Regression on valid set'] = accuracy_score(y_valid, log_predict)

In [19]:
scaler = StandardScaler()
logit = LogisticRegression(random_state=17, class_weight='balanced')

logit_pipe = Pipeline([('scaler', scaler), ('logit', logit)])
logit_pipe_params = {'logit__C': np.logspace(-8, 8, 17)}

Logistic_Regression_grid = GridSearchCV(logit_pipe, param_grid=logit_pipe_params, scoring='accuracy', cv = 5)
Logistic_Regression_grid.fit(X_train, y_train)
print('лучшее значение скора на кросс-валидации -', Logistic_Regression_grid.best_score_)

лучшее значение скора на кросс-валидации - 0.7795031055900621


In [20]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_valid)
dict_model_result['Accuracy knn on valid set'] = accuracy_score(y_valid, knn_pred)

In [21]:
dict_model_result

{'Accuracy Bagging on valid set': 0.8705035971223022,
 'Accuracy Logistic Regression on valid set': 0.8705035971223022,
 'Accuracy Random Forest on valid set': 0.8920863309352518,
 'Accuracy knn on valid set': 0.8129496402877698}

In [22]:
knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])
knn_params = {'knn__n_neighbors': range(3, 15)}
knn_grid = GridSearchCV(knn_pipe, knn_params, cv=5, n_jobs=-1, verbose=True)
knn_grid.fit(X_train, y_train)
knn_grid.best_params_, knn_grid.best_score_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    8.4s finished


({'knn__n_neighbors': 12}, 0.7670807453416149)

In [23]:
#Сначала в качестве ансамбля сделаем простое голосование моделей, а потом стекинг(блендинг)

In [24]:
# majority vote
def Simple_Ensembles(*args):
    Ensembles_predict = []
    arr = np.array(args).T
    for i in range(0, arr.shape[0]):
        one_line = list(arr[i])
        set_one_line = set(one_line)
        
        most_common = None
        qty_most_common = 0

        for item in set_one_line:
            qty = one_line.count(item)
            if qty > qty_most_common:
                qty_most_common = qty
                most_common = item

        Ensembles_predict.append(most_common)
    return Ensembles_predict      

In [25]:
result = Simple_Ensembles(bc_preds, rfc_preds, log_predict, knn_pred)

In [26]:
result = np.array([result])
result = result.reshape((139,))

In [27]:
accuracy = accuracy_score(y_valid, result)
accuracy

0.8705035971223022

In [ ]:
#теперь построим стекинг(блендинг)

In [28]:
X_train, X_valid, y_train, y_valid = train_test_split(df1, y, test_size = 0.3, random_state=17)

In [29]:
X_train_part1 = X_train.reset_index(drop=True).loc[0:220,]
X_train_part2 = X_train.reset_index(drop=True).loc[221:,]

y_train_part1 = y_train.reset_index(drop=True).loc[0:220,]
y_train_part2 = y_train.reset_index(drop=True).loc[221:,]

In [30]:
# bagging over trees
bc = BaggingClassifier(base_estimator=clf_tree, n_estimators=40, random_state=17)
bc.fit(X_train_part1, y_train_part1)
bc_preds_part2 = bc.predict(X_train_part2)
accuracy_score(y_train_part2, bc_preds_part2)                                            

0.8514851485148515

In [31]:
bc_valid = bc.predict(X_valid)
accuracy_score(y_valid, bc_valid) 

0.8848920863309353

In [32]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train_part1, y_train_part1)
knn_pred = knn.predict(X_train_part2)
accuracy_score(y_train_part2, knn_pred)

0.801980198019802

In [33]:
knn_valid = knn.predict(X_valid)
accuracy_score(y_valid, knn_valid) 

0.8129496402877698

In [34]:
logit = LogisticRegression(random_state=17, multi_class='ovr', class_weight='balanced')
logit.fit(X_train_part1, y_train_part1)
log_predict = logit.predict(X_train_part2)
accuracy_score(y_train_part2, log_predict)

0.8118811881188119

In [35]:
log_predict_valid = logit.predict(X_valid)
accuracy_score(y_valid, log_predict_valid) 

0.8705035971223022

In [36]:
rfc = RandomForestClassifier(max_depth=4,  max_features=None, random_state=17)
rfc.fit(X_train_part1, y_train_part1)
rfc_preds_part2 = rfc.predict(X_train_part2)
accuracy_score(y_train_part2, rfc_preds_part2)

0.8910891089108911

In [37]:
rfc_valid = rfc.predict(X_valid)
accuracy_score(y_valid, rfc_valid) 

0.841726618705036

In [38]:
d1 = {'BaggingClassifier_train': bc_preds_part2, 'RandomForestClassifier_train': rfc_preds_part2, \
      'KNeighborsClassifier_train': knn_pred, 'LogisticRegression_train': log_predict}
df1 = pd.DataFrame(data=d1)

d2 = {'BaggingClassifier_test': bc_valid, 'RandomForestClassifier_test': rfc_valid, \
      'KNeighborsClassifier_test': knn_valid, 'LogisticRegression_test': log_predict_valid}
df2 = pd.DataFrame(data=d2)

In [39]:
logit = LogisticRegression(random_state=17, multi_class='ovr', class_weight='balanced')
logit.fit(df1, y_train_part2)
log_predict = logit.predict(df2)
accuracy_score(y_valid, log_predict)

0.841726618705036

In [40]:
# т.к. задача мультиклассовой классификации, мы можем сести ее к построению нескольких бинарных классификаторов  по схемам
# Один против всех (one-versus-all) и Все против всех (all-versus-all)
logit = LogisticRegression(random_state=17, multi_class='ovr', class_weight='balanced')
ovr = OneVsRestClassifier(logit)
ovr.fit(df1, y_train_part2)
Y_pred_ovr = ovr.predict(df2)
ovr_jaccard_score = jaccard_similarity_score(y_valid, Y_pred_ovr)
print(ovr_jaccard_score)

logit = LogisticRegression(random_state=17)
ovo = OneVsOneClassifier(logit)
ovo.fit(df1, y_train_part2)
Y_pred_ovr = ovo.predict(df2)
ovr_jaccard_score = jaccard_similarity_score(y_valid, Y_pred_ovr)
print(ovr_jaccard_score)

logit = LogisticRegression(random_state=17)
occ = OutputCodeClassifier(logit)
occ.fit(df1, y_train_part2)
Y_pred_ovr = occ.predict(df2)
ovr_jaccard_score = jaccard_similarity_score(y_valid, Y_pred_ovr)
print(ovr_jaccard_score)

0.8561151079136691
0.8633093525179856
0.8489208633093526
